In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle



In [48]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [49]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)





In [50]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model


In [51]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)


In [52]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [53]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``m

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-11-02 00:38:15.297820: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.318037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.336528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.339901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.341453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.350092: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-11-02 00:38:15.369432: I tensorflow/core/grappler/optimizers/cust

167/167 [==============================] - 11s 26ms/step - loss: 0.6635 - accuracy: 0.6327
Epoch 2/50
167/167 [==============================] - ETA: 0s - loss: 0.5243 - accuracy: 0.7504Epoch 2/50
Epoch 2/50
167/167 [==============================] - 12s 31ms/step - loss: 0.5385 - accuracy: 0.7508
Epoch 2/50
167/167 [==============================] - 12s 30ms/step - loss: 0.5349 - accuracy: 0.7506
Epoch 2/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4558 - accuracy: 0.8067
Epoch 3/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4649 - accuracy: 0.8001
Epoch 3/50
167/167 [==============================] - 2s 13ms/step - loss: 0.4767 - accuracy: 0.7979
Epoch 3/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4400 - accuracy: 0.8108
Epoch 3/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4376 - accuracy: 0.8102
Epoch 3/50
167/167 [==============================] - 2s 14ms/step - loss: 0.4324 - accuracy: 

...

/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 2s 14ms/step - loss: 0.4633 - accuracy: 0.7973
Epoch 3/100
167/167 [==============================] - 3s 15ms/step - loss: 0.4743 - accuracy: 0.8031
Epoch 3/100
167/167 [==============================] - 3s 15ms/step - loss: 2.2727 - accuracy: 0.7518
Epoch 44/50
167/167 [==============================] - 2s 14ms/step - loss: 0.4323 - accuracy: 0.8108
Epoch 5/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4439 - accuracy: 0.8057
Epoch 3/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4380 - accuracy: 0.8060
Epoch 4/100
167/167 [==============================] - 4s 15ms/step - loss: 0.5593 - accuracy: 0.7302
Epoch 2/100
167/167 [==============================] - 2s 15ms/step - loss: 2.6307 - accuracy: 0.7537
Epoch 45/50
167/167 [==============================] - 2s 14ms/step - loss: 0.4319 - accuracy: 0.8114
Epoch 4/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4340 - accuracy:

/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 2s 14ms/step - loss: 0.4306 - accuracy: 0.8117
Epoch 7/100
167/167 [==============================] - 2s 15ms/step - loss: 2.9819 - accuracy: 0.7463
Epoch 46/50
167/167 [==============================] - 2s 14ms/step - loss: 0.4338 - accuracy: 0.8082
Epoch 6/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4357 - accuracy: 0.8099
Epoch 6/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4303 - accuracy: 0.8091
Epoch 8/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4352 - accuracy: 0.8073
Epoch 4/100
167/167 [==============================] - 2s 15ms/step - loss: 0.4309 - accuracy: 0.8101
Epoch 6/100
167/167 [==============================] - 3s 15ms/step - loss: 0.4340 - accuracy: 0.8041
Epoch 5/100
167/167 [==============================] - 3s 15ms/step - loss: 0.4307 - accuracy: 0.8117
Epoch 7/100
167/167 [==============================] - 2s 15ms/step - loss: 0.4336 - accuracy:

KeyboardInterrupt: 